# Lab20: Association Analysis

In this lab we will be working on identifying *association rules*.  Association a way of identifying a combination of items that are associated with other items in the form of an IF-THEN type of rule:

    X -> Y

where X and Y are sets of items (goods, words, etc.).  The rule above asks if we purchase X then how often do we also purchase Y?

## Install mlxtend

We will be using the package `mlxtend` for our analysis.  This is a machine learning package (hence the ml) and has some features that overlap with SciKit Learn and other analysis pacakages.  

Using `Anaconda Navigator` install the `mlxtend` package.  It is on the `conda-forge` channel, so you will have to make sure that you have that set up as well.  (We did this back in Lab01.)

In [1]:
import pandas as pd

# TransactionEncoder will convert our data set to something
# the association-rule algorithm can use
from mlxtend.preprocessing import TransactionEncoder

# We will use the apriori module to generate a dataframe that
# we can use for association rule finding
from mlxtend.frequent_patterns import apriori

# We will use the association_rules module to generate
# our association rules from the apriori output data frame
from mlxtend.frequent_patterns import association_rules

## Load our groceries data set

This is from [Kaggle](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset?resource=download).  It is a CSV file containing around 38000 purchases of people from grocery stores.

Load and examine the file as a Pandas DataFrame.

Use the `sort_values` method to sort first by CustomerID and then by date, and examine the resulting sorted DataFrame. You will note that the file appears to contain daily transactions for each customer.  We will assume that a customer's daily purchases constitute a single transaction.

In [2]:
fn_grocery = 'Groceries_dataset.csv'
df_raw = pd.read_csv(fn_grocery)

In [4]:
df_raw.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


## Group the transactions

Using `groupby` group each transaction by `Member_number` and `Date`.  This will give us one "row" with each purchase per customer per day.  We will use this in a moment to encode a new DataFrame for our `mlxtend`'s use.

In [10]:
df_grouped = df_raw.groupby(['Member_number', 'Date'])
df_grouped


## Encode the grouped data frame

Here we need to transform our dataset with the `Member_number` and `Date` index into a DataFrame that `mlxtend` can use to calculate the support of each transaction.

The `agg` method will return a DataFrame.  We need to convert its output into a list of lists.

In [11]:
def make_transaction_list(x):
    '''
    Will return a list of the unique items
    in a particular grouping when used with
    the agg method as its function
    '''
    return x.unique()


In [23]:
# Use the agg method and make_transaction_list
# to return a list of unique items for each
# transaction
df_transactions = df_grouped.agg(make_transaction_list)
df_transactions.columns

Index(['itemDescription'], dtype='object')

## Convert to list of lists

From the DataFrame you created using the `agg` function, turn the output Series into a list of lists.

In [ ]:
list_trans = list(df_transactions['itemDescription'])
list_trans

## Encode the lists

Use the [`TransactionEncoder` method](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/) from `mlxtend` to encode our list of lists into a matrix of presence/absence Boolean values.  See the example at the link above.

Once you do that, we can re-convert the output into a DataFrame by creating a dataframe based on the output and the value in the TransactionEncoder's `column_` attribute.  Again, see the link above.  It shows how to do this.

In [40]:
te = TransactionEncoder()
encoded_itemset = te.fit(list_trans).transform(list_trans)
encoded_itemset.shape  # Transactions, # of items
te.columns_  # These are the different items

# Let's make a data frame from our encoded itemset
df_encoded = pd.DataFrame(encoded_itemset, columns=te.columns_)
df_encoded.head()



,,(,),-,/,H,Instant food products,T,U,UHT-milk,...,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,y,yogurt,z,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Use `apriori` to get support

The input to the association rule generator is a data frame that contains support for each transaction.  The [`aprori`](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) module will do this for us.  See the link for documentation.

Let's use

    min_support=0.001  # All transactions that are > 1:1000

In [41]:
# Before we begin, let's do a small
# amount of cleanup.  Let's remove all
# columns (items) that have a name less
# than three letters.  Use the Pandas
# drop method to perform this action.
# Let's drop columns with labels less than 3 characters
# as they don't appear to be meaningful
small_items = list(filter(lambda x: len(x) < 3, te.columns_ ))
df_encoded = df_encoded.drop(columns=small_items)
print('How many unique items are left?', len(df_encoded.columns))

How many unique items are left? 167


In [45]:
# Use apriori to create a dataframe with columns
# of support and itemset lists
df_support = apriori(df_encoded, min_support=0.001, use_colnames=True)
df_support.sort_values('support', inplace=True, ascending=False)
df_support

,support,itemsets
146,0.155250,(whole milk)
90,0.120831,(other vegetables)
109,0.108401,(rolls/buns)
123,0.096037,(soda)
147,0.084876,(yogurt)
...,...,...
344,0.001002,"(margarine, chicken)"
201,0.001002,"(bottled beer, chicken)"
202,0.001002,"(bottled beer, chocolate)"
516,0.001002,"(hamburger meat, pastry)"


## Use association_rules to find the rules

Using the dataframe generated by `apriori`, find the association rules with the greatest lift.  See the [association_rules documentation](https://rasbt.github.io/mlxtend/api_modules/mlxtend.frequent_patterns/association_rules/) for how to do this.

Sort the resulting DataFrame by lift in descending order.  A lift > 1 indicates that the items are often purchased together and that buying X will increase the purchase of Y.  A lift of < 1 indicates the items are often substituted.  That is X is substituted for Y so X and Y don't appear together often.

Examine the resulting DataFrame.  For the association rule X -> Y, X is the column `antecedents` and Y is the column `consequents`.  If sorted you can see the metrics for each rule based upon the lift.

In [49]:
# Find the association rules
rules = association_rules(df_support, metric='lift', min_threshold=1.0)
# lift > 1  (complimentation) more likely than chance X means you buy Y
# lift = 1  as often as chance that X means you buy Y
# lift < 1  (substitution) less likely than chance X means you buy Y_value
rules.sort_values('lift', ascending=False, inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
93,(sausage),"(whole milk, yogurt)",0.060215,0.011161,0.001470,0.024417,2.187762,0.000798,1.013588
92,"(whole milk, yogurt)",(sausage),0.011161,0.060215,0.001470,0.131737,2.187762,0.000798,1.082373
95,(yogurt),"(sausage, whole milk)",0.084876,0.008955,0.001470,0.017323,1.934340,0.000710,1.008515
90,"(sausage, whole milk)",(yogurt),0.008955,0.084876,0.001470,0.164179,1.934340,0.000710,1.094880
110,(citrus fruit),(specialty chocolate),0.052864,0.015973,0.001403,0.026549,1.662125,0.000559,1.010864
...,...,...,...,...,...,...,...,...,...
69,(domestic eggs),(bottled beer),0.036891,0.045044,0.001671,0.045290,1.005448,0.000009,1.000257
182,(sausage),(salty snack),0.060215,0.018780,0.001136,0.018868,1.004700,0.000005,1.000090
183,(salty snack),(sausage),0.018780,0.060215,0.001136,0.060498,1.004700,0.000005,1.000301
50,(oil),(other vegetables),0.014903,0.120831,0.001804,0.121076,1.002026,0.000004,1.000279


In [ ]:
# Sort the rules by lift
# and examine the output
# to find what rules were
# discovered
